<a href="https://colab.research.google.com/github/jamesbaskerville/colabs/blob/main/WikipediaCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this exploratory project is to crawl English Wikipedia and get a shortest path from source to target, e.g. from [Computer science](https://en.wikipedia.org/wiki/Computer_science) to [Harry R. Lewis](https://en.wikipedia.org/wiki/Harry_R._Lewis).


There are two main approaches, utilizing asyncio workers for parallelism:
1. BFS using a priority queue ordered by length of path. Basically just doing a naive search using Wikipedia's graph network of links.
2. BFS where the priority score is determined by embedding similarity of each article's content to the target article. The idea is that higher similiarity candidates for exploration will better direct our search.

In practice, (1) still performs much faster as the API calls for content and model embeddings take significantly longer than each naive BFS operation. However, (2) does find a path while visiting fewer overall pages, giving some credence to the idea that the search is more directed (albeit slower at each node).

## Scratchpad


- will need to build adjacency graph or at least figure out how to queue links
- single hop needs how to extract links
- this probably looks like beautiful soup for HTML parsing
  - simple = find all anchors, filter by ones that start with en.wikipedia.com/wiki
- parsers should probably be
- where is the content stored? is there a text version of wikipedia

#### API
https://en.wikipedia.org/w/api.php?action=parse&page=Computer%20science&prop=links&format=json
wikipedia has an api that can easily pull links from an article. Maybe good place to start to avoid HTML parsing for now. Just need JSON parsing.

## Code

In [1]:
%%capture
!pip install aiohttp
!pip install sentence-transformers

In [33]:
import asyncio
import aiohttp
import time
import logging

class RateLimiter:
    def __init__(self, requests_per_second):
        self.requests_per_second = requests_per_second
        self.last_request_time = 0

    async def wait_for_rate_limit(self):
        current_time = time.time()
        time_since_last_request = current_time - self.last_request_time
        if time_since_last_request < 1 / self.requests_per_second:
            await asyncio.sleep(1 / self.requests_per_second - time_since_last_request)
        self.last_request_time = time.time()

class WikipediaClient:
  def __init__(self, rate_limit=5):
    # include rate limiting
    self.rate_limiter = RateLimiter(5)

  async def fetch_page(self, page: str, prop: str):
    async with aiohttp.ClientSession() as session:
      await self.rate_limiter.wait_for_rate_limit()
      if prop == 'content':
        url = f'https://en.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&explaintext&titles={page}'
      else:
        url = f'https://en.wikipedia.org/w/api.php?action=parse&page={page}&props=links&format=json'

      logging.debug(f"Fetching {url}")
      async with session.get(url) as response:
        response = await response.json()

      if prop == 'content':
        logging.debug(f"Fetched content for {url}")
        try:
          return '\n'.join([x['extract'] for x in response['query']['pages'].values()])
        except:
          print(f"Error: {response}")
          return ""
      else:
        logging.debug(f"Fetched links for {url}")
        logging.debug(f"Response: {response}")
        return [x["*"] for x in filter(lambda x: x['ns'] == 0, response['parse']['links'])]

  async def get_adjacent_pages(self, page: str):
    page_names = await self.fetch_page(page, 'links')
    return page_names

  async def get_page_content(self, page: str):
    return await self.fetch_page(page, 'content')

In [4]:
%%capture
from sentence_transformers import SentenceTransformer
from sentence_transformers import util


class SentenceEmbedder:
  def __init__(self):
    self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

  def embed(self, text):
    return self.model.encode(text, convert_to_tensor=True)

  # 0 to 1 with 0 being most similar
  def similarity(self, embedding1, embedding2):
    cos_sim = util.cos_sim(embedding1, embedding2).item()
    # normalize
    return (-1 * cos_sim + 1) / 2

test_embedder = SentenceEmbedder()
test_embedder.similarity(test_embedder.embed("hello how are you"), test_embedder.embed("i really dislike you"))

In [28]:
import traceback
import random

# Naive way to search for target = BFS
# Idea = use semantic comparison somehow to word2vec or otherwise
# compare current & target and use a priority queue BFS
# should also store path and update in in maybe a dictionary instead
# of a visited set; e.g. store shortest path to a node in the dict value
class WikipediaCrawler:
  def __init__(
        self,
        start_page: str,
        target='Kevin Bacon',
        max_concurrent=5,
        rate_limit=5,
        max_depth=5,
        strategy="depth",
        debug=False
    ):
    self.start_page = start_page
    self.target = target
    self.queue = asyncio.PriorityQueue()
    self.visited = {}
    self.embeddings = {}
    self.errored = set()
    self.max_concurrent = max_concurrent
    self.rate_limit = rate_limit
    self.max_depth = max_depth
    self.stop_event = asyncio.Event()
    self.strategy = strategy
    self.embedder = SentenceEmbedder()
    self.client = WikipediaClient(rate_limit)
    self.workers = []
    self.debug = debug

  async def priority_score(self, page, prev):
    if self.strategy == "depth":
      return len(self.visited[prev]) if prev in self.visited else 0
    elif self.strategy == "embedding":
      if self.target not in self.embeddings:
        target_content = await self.client.get_page_content(self.target)
        self.embeddings[self.target] = self.embedder.embed(target_content)

      if page not in self.embeddings:
        page_content = await self.client.get_page_content(page)
        if not page_content:
          return 99.0
        self.embeddings[page] = self.embedder.embed(page_content)
      result = self.embedder.similarity(self.embeddings[page], self.embeddings[self.target])
      return result
    else:
      return 99.0


  async def crawl(self):
    await self.queue.put((0, self.start_page, None)) # depth/priority, page, prev
    self.workers = [asyncio.create_task(self.worker(f'worker{i}')) for i in range(self.max_concurrent)]

    try:
      await asyncio.gather(*self.workers)
      print('Done with workers')
      for worker in self.workers:
        worker.cancel()
    except asyncio.CancelledError:
        print("Tasks cancelled.")
    finally:
        self.stop_event.set()  # Signal workers to stop
        # runtime.unassign()  # Terminate Colab runtime

    if self.target in self.visited:
      print(f"Found target page {self.target} in {len(self.visited[self.target])} hops")
      print(f"Path: {self.visited[self.target]}")
      print(f"Total pages visited: {len(self.visited)}")
      return True
    else:
      print(f"Did not find target page {self.target}")
      print(f"Total pages visited: {len(self.visited)}")
      return False

  async def worker(self, workerName):
    if self.debug:
        print(f"Starting {workerName}")

    while not self.stop_event.is_set():
      prio_score, page, prev = await self.queue.get()
      if self.debug:
        print(f"Queue length: {self.queue.qsize()}; processing {page} with score {prio_score}")

      depth = len(self.visited[prev]) if prev else 0
      if depth > self.max_depth:
        self.queue.task_done()
        return

      # update to shorter path if possible
      if page in self.visited and len(self.visited[prev]) + 1 < len(self.visited[page]):
        self.visited[page] = self.visited[prev] + [prev]
        self.queue.task_done()
        continue

      if page not in self.visited:
        if not prev:
          self.visited[page] = [page]
        else:
          self.visited[page] = self.visited[prev] + [page]

        if self.strategy == 'embedding' or len(self.visited) % 200 == 0:
          print(f"Processed {len(self.visited)} pages, currently at depth {depth}, queue length {self.queue.qsize()}")

        try:
          adjacent_pages = await self.client.get_adjacent_pages(page)

          if self.debug:
            print(f"Found {len(adjacent_pages)} adjacent pages for {page}")

          # for the embeddings approach iterating through everything is quite heavy
          # maybe should just sample some and move on
          for adjacent_page in adjacent_pages:
            if self.stop_event.is_set():
              if self.debug:
                print(f"Stopping {workerName}")
              self.queue.task_done()
              return

            # Success!
            if adjacent_page == self.target:
              self.visited[self.target] = self.visited[prev] + [page, adjacent_page]
              self.queue.task_done()
              print(f"Found target with path: {self.visited[self.target]}")
              self.stop_event.set()

            elif adjacent_page not in self.visited and depth + 1 <= self.max_depth:
              new_prio_score = await self.priority_score(adjacent_page, prev)
              if self.debug:
                print(f"Adding {adjacent_page} with priority {new_prio_score}")
              await self.queue.put((new_prio_score, adjacent_page, page))
            else:
              pass
        except Exception as e:
          if self.debug:
            print(f"Error processing {page}: {e}")
            traceback.print_exc()
          self.errored.add(page)
          # retry once
          if page not in self.errored:
            if self.debug:
              print(f"Retrying {page}")
            await self.queue.put((prio_score, page, prev))

      self.queue.task_done()
    print(f"{workerName} exiting gracefully.")
    return

In [27]:
# Use BFS alone -- order priority queue by length of path
crawler = WikipediaCrawler(
    'Computer science',
    target="Singapore",
    rate_limit=5,
    max_depth=10,
    strategy="depth"
  )
await crawler.crawl()

Processed 5 pages, currently at depth 1, queue length 556
Processed 10 pages, currently at depth 1, queue length 1687
Processed 15 pages, currently at depth 1, queue length 3897
Processed 20 pages, currently at depth 1, queue length 5010
Processed 25 pages, currently at depth 1, queue length 6506
Processed 30 pages, currently at depth 1, queue length 7089
Found target with path: ['Computer science', 'Artificial intelligence', 'Singapore']
Done with workers
Found target page Singapore in 3 hops
Path: ['Computer science', 'Artificial intelligence', 'Singapore']
Total pages visited: 34


True

In [34]:
# Use text embeddings of article content to guide search
crawler = WikipediaCrawler(
    'Computer science',
    target="Singapore",
    rate_limit=5,
    max_depth=2,
    strategy="embedding"
  )
await crawler.crawl()

Processed 1 pages, currently at depth 0, queue length 0
Processed 2 pages, currently at depth 1, queue length 0
Processed 3 pages, currently at depth 1, queue length 0
Processed 4 pages, currently at depth 1, queue length 0
Processed 5 pages, currently at depth 1, queue length 0
Processed 6 pages, currently at depth 1, queue length 3
Processed 7 pages, currently at depth 2, queue length 7
Processed 8 pages, currently at depth 2, queue length 13
Processed 9 pages, currently at depth 2, queue length 19
Processed 10 pages, currently at depth 1, queue length 26
Error: {'batchcomplete': '', 'query': {'pages': {'-1': {'ns': 0, 'title': 'List of important publications in theoretical computer science', 'missing': ''}}}}
Processed 11 pages, currently at depth 2, queue length 1369
Processed 12 pages, currently at depth 2, queue length 1373
Found target with path: ['Computer science', 'History of computing', 'China', 'Singapore']
Done with workers
Found target page Singapore in 4 hops
Path: ['Com

True